In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import get_cmap
cm = get_cmap('jet')

import imgaug.augmenters as iaa
from keras.models import load_model

In [ ]:
class_categories = {'Branching' : 0, 
                  'Fish' : 1, 
                  'Massive' : 2,
                  'Not Massive' : 3,
                  'Substrate' : 4,
                  'Target' : 5,
                  'Water' : 6}

In [ ]:
image_file = "..\\Figures\\example_coral_patch.png"
image_name = os.path.basename(image_file).split(".")[0]

image = plt.imread(image_file)   

In [ ]:
patch_df = pd.read_csv(image_name + "_Sparse_Points.csv")

patches = []

for i, r in patch_df.iterrows():
    
    patch = plt.imread(r['file'])
    patch = iaa.Resize(224).augment_image(image=patch) * (1./255.0)
    patches.append(patch)

patches = np.array(patches)

In [ ]:
# Load up the best trained model
model = load_model("Best_Model_and_Weights.h5")

In [ ]:
# Making predictions on the patches that were extracted
predictions = model.predict(patches)

# Getting the predicted labels and also the confidence score between the top two likely classes
predicted_labels = [list(class_categories)[np.argmax(prediction, axis = 0)] for prediction in predictions]
confidence = [sorted(prediction)[-1] - sorted(prediction)[-2] for prediction in predictions]

# Adding the to the dataframe
patch_df['Label'] = predicted_labels
patch_df['Confidence'] = confidence

# Saving again
patch_df.to_csv(image_name + "_Sparse_Points.csv")

In [ ]:
# Just viewing the image with the sparse labels super imposed
additional_labels = patch_df[patch_df['Confidence'] >= .70]

X = additional_labels['X'].values
Y = additional_labels['Y'].values
C = [cm[_]/255.0 for _ in additional_labels['Labels'].values]

plt.figure(figsize = (10, 10))
plt.imshow(image)
plt.scatter(X, Y, c = C)
plt.xticks([])
plt.yticks([])